In [30]:
import pandas as pd

# 读取 Excel 文件
data = pd.read_excel(r"W:\材智科技\中铝\测试数据.xlsx")

In [37]:
data.iloc[:-10,:4].to_numpy()

array([[0.000e+00, 4.741e+00, 3.050e+02, 5.200e+02],
       [3.700e-02, 4.807e+00, 3.050e+02, 5.200e+02],
       [3.700e-02, 4.943e+00, 3.050e+02, 5.200e+02],
       ...,
       [3.600e-01, 5.700e+00, 2.850e+02, 4.800e+02],
       [3.600e-01, 5.700e+00, 2.850e+02, 4.800e+02],
       [3.600e-01, 5.700e+00, 2.850e+02, 5.000e+02]])

In [38]:
data.iloc[:-10,4:].to_numpy()

array([[117],
       [120],
       [128],
       [116],
       [116],
       [117],
       [117],
       [115],
       [113],
       [118],
       [129],
       [126],
       [138],
       [128],
       [120],
       [121],
       [127],
       [129],
       [121],
       [124],
       [128],
       [127],
       [122],
       [117],
       [108],
       [123],
       [116],
       [130],
       [123],
       [125],
       [105],
       [120],
       [135],
       [122],
       [121],
       [112],
       [120],
       [122],
       [128],
       [122],
       [122],
       [126],
       [121],
       [122],
       [135],
       [122],
       [124],
       [125],
       [122],
       [118],
       [134],
       [118],
       [121],
       [110],
       [124],
       [121],
       [127],
       [128],
       [128],
       [129],
       [126],
       [134],
       [135],
       [123],
       [116],
       [118],
       [130],
       [134],
       [111],
       [101],
       [113],
      

In [57]:
import numpy as np

alpha = 0.001
hidden_size_1 = 32
hidden_size_2 = 16
epoch = 1000

layer_0_1_weights = 2 * np.random.random((4, hidden_size_1)) - 1
layer_1_2_weights = 2 * np.random.random((hidden_size_1, hidden_size_2)) - 1
layer_2_3_weights = 2 * np.random.random((hidden_size_2, 1)) - 1


def relu(x):
    return (x > 0) * x


def relu_origin(x):
    return x > 0

# 计算R²值的函数
def calculate_r2(y_true, y_pred):
    """
    计算R²（决定系数）
    R² = 1 - (SSE / SST)
    其中：
    SSE = Sum of Squared Errors = Σ(y_true - y_pred)²
    SST = Total Sum of Squares = Σ(y_true - y_mean)²
    """
    # 确保输入是numpy数组
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()

    # 计算真实值的均值
    y_mean = np.mean(y_true)

    # 计算SSE（残差平方和）
    sse = np.sum((y_true - y_pred) ** 2)

    # 计算SST（总平方和）
    sst = np.sum((y_true - y_mean) ** 2)

    # 计算R²
    if sst == 0:
        return 1.0 if sse == 0 else 0.0

    r2 = 1 - (sse / sst)
    return r2

# street_lights = np.array([
#     [0, 4.741, 305, 520],
#     [0.037, 4.807, 305, 520],
#     [0.037, 4.943, 305, 520],
#     [0.037, 4.943, 305, 520],
#     [0.037, 4.807, 305, 520],
#     [0.006, 4.918, 305, 520],
#     [0.037, 4.943, 305, 520],
#     [0.037, 4.807, 305, 520],
#     [0.041, 3.48, 320, 500],
#     [0.022, 4.87, 305, 520]
# ])
street_lights = data.iloc[:-10,:4].to_numpy()

# 计算每列的均值和标准差
inputs_mean = np.mean(street_lights, axis=0)
inputs_std = np.std(street_lights, axis=0)

# 标准化
street_lights = (street_lights - inputs_mean) / inputs_std
# street_lights = street_lights/100

# walk_stop = np.array([
#     [117],
#     [120],
#     [128],
#     [116],
#     [116],
#     [117],
#     [117],
#     [115],
#     [113],
#     [118]
# ])
walk_stop = data.iloc[:-10,4:].to_numpy()

# 计算每列的均值和标准差
outputs_mean = np.mean(walk_stop, axis=0)
outputs_std = np.std(walk_stop, axis=0)

# 标准化
walk_stop = (walk_stop - outputs_mean) / outputs_std
# walk_stop = walk_stop/100

for n in range(epoch):
    all_error = 0
    predictions = []
    for i in range(len(street_lights)):
        inputs = street_lights[i]
        layer_1 = relu(inputs.dot(layer_0_1_weights))
        layer_2 = relu(layer_1.dot(layer_1_2_weights))
        layer_3 = layer_2.dot(layer_2_3_weights)

        error = np.sum((layer_3 - walk_stop[i]) ** 2)

        layer_3_delta = layer_3 - walk_stop[i]
        layer_2_delta = layer_3_delta.dot(layer_2_3_weights.T) * relu_origin(layer_2)
        layer_1_delta = layer_2_delta.dot(layer_1_2_weights.T) * relu_origin(layer_1)

        layer_3_delta_weight = layer_2.reshape(hidden_size_2, 1).dot(layer_3_delta.reshape(1, 1))  # (10, 1)
        layer_2_delta_weight = layer_1.reshape(hidden_size_1, 1).dot(layer_2_delta.reshape(1, hidden_size_2))  # (10, 10)
        layer_1_delta_weight = inputs.reshape(4, 1).dot(layer_1_delta.reshape(1, hidden_size_1))  # (4, 10)

        layer_2_3_weights -= alpha * layer_3_delta_weight
        layer_1_2_weights -= alpha * layer_2_delta_weight
        layer_0_1_weights -= alpha * layer_1_delta_weight
        all_error += error
        predictions.append(layer_3)
        if (n % 100 == 0 and i < 10) or (n == epoch - 1 and i < 10):
            print('Predict:', layer_3 * outputs_std + outputs_mean)
            # print('Predict:', layer_3 * 100)
    if n % 100 == 0 or n == epoch - 1:
        print('R²:', calculate_r2(walk_stop, predictions))
        print('Error:', all_error)

Predict: [159.69265871]
Predict: [157.69223653]
Predict: [157.46220353]
Predict: [155.58606143]
Predict: [151.62936827]
Predict: [149.40803582]
Predict: [149.45018399]
Predict: [146.514332]
Predict: [241.77566513]
Predict: [140.32556617]
R²: -3.8263544401460416
Error: 1949.8471938190012
Predict: [117.52600236]
Predict: [120.00302169]
Predict: [119.68027345]
Predict: [119.95430253]
Predict: [120.13556092]
Predict: [119.12309327]
Predict: [119.6385089]
Predict: [119.88436819]
Predict: [102.44401679]
Predict: [120.92628597]
R²: 0.5678081666168899
Error: 174.6055006867765
Predict: [116.90910744]
Predict: [119.96102869]
Predict: [119.6343533]
Predict: [119.92018433]
Predict: [120.10239187]
Predict: [119.30047988]
Predict: [119.5925036]
Predict: [119.84183589]
Predict: [102.74264434]
Predict: [120.83576906]
R²: 0.5745435658640461
Error: 171.88439939092532
Predict: [116.36930043]
Predict: [120.0030087]
Predict: [119.66437925]
Predict: [119.9668541]
Predict: [120.1492493]
Predict: [119.3916569

In [16]:
a = np.array([[1], [0], [1]])
b = np.array([[1.1, 2.1, 3.1, 4.1]])

print('a:', a)
print('b:', b)
print(a.dot(b))

a: [[1]
 [0]
 [1]]
b: [[1.1 2.1 3.1 4.1]]
[[1.1 2.1 3.1 4.1]
 [0.  0.  0.  0. ]
 [1.1 2.1 3.1 4.1]]


In [13]:
len(street_lights)

10